In [ ]:
import warnings
warnings.filterwarnings("ignore")
import xarray as xr
xr.set_options(display_style='html')

# Xarray for the geosciences

*Raphael Dussin, April 2021*

![](http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png "lgo")

# Outline

### 1. Numerical models of the ocean
### 2. Working with ocean model data: numpy
### 3. Xarray/Dask improved workflow

### What is the ocean made of?

<img src=https://i.pinimg.com/originals/b0/26/1e/b0261e1bbd7010bfb88c982cf19ff5ad.jpg width=800>

If we could see the ocean at the nanometer scale, we would see a soup of molecules (mostly H20) moving around. The speed of the molecules and volume occupied by them depends on the temperature, or rather temperature is the macroscopic measurement of the microscopic agitation of molecules. In a solid (like ice), molecules are tightly packed and bond electromagnetically which gives the solid its rigidity. For the ocean it is impossible to measure the individual movement of molecules and the observer has to consider this soup of molecules as a "continuous media", looking at the macroscopic properties of the fluid such as its temperature and concentration in salt and other molecules.

### Newton second's law is that: 

### mass x acceleration = sum of all forces

<img src=https://d2r55xnwy6nx47.cloudfront.net/uploads/2018/01/Navier-StokesEquation_560.jpg width=800>

<img src=https://images.slideplayer.com/16/5261667/slides/slide_7.jpg width=800>

### Unfortunately these equations are impossible to solve by hand, needs a computer

### convert continuous operators (e.g. $\partial_{t} u$) into discrete operators (e.g. $(u(t + \Delta t) - u(t))/\Delta t$)

### all numerical schemes are only accurate up to a certain order!

### higher order are better but more computationally expensive

<img src=https://image2.slideserve.com/4507757/grid-boxes-in-a-three-dimensional-ocean-model-l.jpg width=800>

Ocean dynamics spans multiple order of magnitude in time and space

<img src=https://www.researchgate.net/profile/Justyna_Jonca/publication/278641703/figure/fig13/AS:669537305382929@1536641541904/11-Spatial-and-temporal-scales-of-ocean-processes-Dickey-2001_W640.jpg width=600>

Ocean model has:

* dx = dy = 1-100km horizontal resolution
* dt=10-1000 seconds (depends on dx)
* runs on supercomputer with O(10) - O(10000) compute cores
* produces GB -> TB of data

File format for geophysical data:

* binary format (ASCII text file is not storage efficient)
* usually self-documented (metadata header + data records)
* some sort of lossless data compression (gzip, blosc,...)
* examples: netCDF, HDF4/5, grib

Data representation is typically 3d/4d arrays:

* time + 2/3 spatial dimensions (longitude, latitude, [depth/pressure])

In [ ]:
#!wget http://35.188.34.63:8080/thredds/fileServer/OM4p5/ocean_monthly_z.200301-200712.nc4

In [ ]:
#!wget https://github.com/raphaeldussin/MOM6-AnalysisCookbook/raw/master/docs/notebooks/data/ocean_grid_sym_OM4_05.nc

#### Working with netcdf files (the old and long way)

* open the file
* inspect for variables
* load variables into a numpy array
* perform operation on the array

In [ ]:
import numpy as np
import netCDF4
nc = netCDF4.Dataset("ocean_monthly_z.200301-200712.nc4")
print(nc.variables.keys())

In [ ]:
thetao = nc["thetao"]
print(type(thetao))

In [ ]:
# thetao contains all the information from the netcdf file 
# includig dimensions, units,...
thetao

In [ ]:
# we can access every part of the metadata
print(thetao.units)

In [ ]:
# including dimensions
print(thetao.dimensions)

now how do we work with the data?

In [ ]:
# we transfer the content of the file records into 
# a new variable (load into memory)
%time data = thetao[:]

In [ ]:
print(type(data))
print(data.shape)

In [ ]:
print(data.units)

In [ ]:
print(data.dims)

What went wrong? we seem to have lost all the metadata between the netcdf4.variable and numpy.array types

To use the data, the scientist must know what the different dimensions refer to and in which order they are. This is error-prone and writing code with numpay.array is not very readable for the non-expert:

In [ ]:
something = data[:,0,:,:].mean(axis=0)
print(type(something))
print(something.shape)
#print(data.shape)

something is the time-averaged Sea Surface Temperature. can you guess from the code?

Now this:

In [ ]:
newthing = data[:,0,:,:].mean(axis=0).mean(axis=1)

In [ ]:
sst_tavg = data[:,0,:,:].mean(axis=0)
print(sst_tavg.shape)
sst_tavg_xavg = sst_tavg.mean(axis=1) # the axis number has shift!!
print(sst_tavg_xavg.shape)
print(np.allclose(newthing, sst_tavg_xavg))

Plotting results:

In [ ]:
import matplotlib.pyplot as plt
grid = netCDF4.Dataset("ocean_grid_sym_OM4_05.nc")
lon = grid["geolon"][:] ; lat = grid["geolat"][:]
plt.figure(figsize=[12, 4])
plt.pcolormesh(lon, lat, sst_tavg, cmap='gist_ncar')
plt.colorbar() ; plt.show()

In [ ]:
del data

confused? we all were!

### xarray to the rescue

* xarray will bridge the gap between the numpy.array and the metadata of the netcdf.variable!
* xarray has interfaces to netcdf files but not only (also grib, zarr, iris,...) and several engines (netcdf4, pydap, scipy.IO,...)

In [ ]:
import xarray as xr

### Xarray dataset:

In [ ]:
ds = xr.open_dataset("ocean_monthly_z.200301-200712.nc4")

Here I open a netcdf file but it could be a collection of files, various format, local OR distant!

In [ ]:
ds

### Xarray DataArray:

In [ ]:
ds["thetao"]

What is under the hood? numpy or dask array

In [ ]:
ds = xr.open_dataset("ocean_monthly_z.200301-200712.nc4")

In [ ]:
print(type(ds['zos']))
print(type(ds['zos'].data)) # takes 1-2 second

In [ ]:
# notice the chunks argument
ds = xr.open_dataset("ocean_monthly_z.200301-200712.nc4",
                     chunks={'time':1})

In [ ]:
print(type(ds['zos']))
print(type(ds['zos'].data)) # instant!

dask array are lazy, numpy array are eager!

merging datasets and playing with dimensions and coordinates is easy

In [ ]:
grid = xr.open_dataset("ocean_grid_sym_OM4_05.nc")
ds["geolon"] = grid["geolon"]
ds["geolat"] = grid["geolat"]

In [ ]:
ds = ds.set_coords(["geolon", "geolat"])

xarray has knowledge of labels such as dimensions so it makes the code more readable!

```python
thetao[:,0,:,:].mean(axis=0)
```

becomes

In [ ]:
sst_tavg = ds["thetao"].sel(z_l=10.).mean(dim='time')

In [ ]:
sst_tavg

at this point, it is all virtual, no data has been computed yet!
Dask built a "graph" (think recipe) of how to obtain the result. 

To get the result of the computation, we need to explicitly load or ask to display the values

In [ ]:
%time sst_tavg.load()

In [ ]:
sst_tavg.plot(figsize=[12,4], cmap='gist_ncar',
              x='geolon', y="geolat", vmin=-2, vmax=32)

Distributed computing with dask

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
client

make a relatively large computation so we can see what's going on with dask:

In [ ]:
Tclim = ds['thetao'].mean(dim='time')

In [ ]:
%time Tclim.load()

In [ ]:
client.close()
cluster.close()

Equivalent computation with numpy

In [ ]:
def numpy_mean():
    nc = netCDF4.Dataset("ocean_monthly_z.200301-200712.nc4", 'r')
    data = nc["thetao"][:]
    mean = data.mean(axis=0)
    return mean

In [ ]:
%time Tclim = numpy_mean()

dask can speed up the computation by distributing on multiple cores but this comes with a price (overhead), this is particularly suited for very large computation that do not fit into memory.